# Competição DSA 2019/06
<h3>Objetivo: Prever Índice de Lealdade</h3>
<h4>Informações gerais disponíveis no <a href="https://www.kaggle.com/t/84f9cb851e9a428fa9fe54644df35f5c">Kaggle</a> </h4>
<h4>Parte 2: processo de ML</h4>

In [1]:
# Importar as bibliotecas necessárias para este notebook
import pandas as pd
import numpy  as np
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Importar os arquivos criados na parte 1
df_teste      = pd.read_csv('df_teste.csv')
df_treino     = pd.read_csv('df_treino.csv')

<h3>Ajustes finais para o processo de ML</h3>

In [3]:
df_treino.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,alvo,oldest_Activation,latest_Activation
0,C_ID_92a2005557,0,0,0,0,1,0,1,0,1,-0.820283,0.071233,0.909589
1,C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,0.392913,0.013699,1.241096
2,C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,0.688056,0.446575,1.739726
3,C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,0.142495,0.068493,0.624658
4,C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,-0.159749,0.030137,0.487671


In [4]:
df_teste.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,oldest_Activation,latest_Activation
0,C_ID_0ab67a22ab,0,0,1,0,0,0,0,1,1,0.008219,0.912329
1,C_ID_130fd0cbdd,0,1,0,0,0,0,0,1,0,0.032877,1.298630
2,C_ID_b709037bc5,0,0,0,0,1,1,0,0,1,0.065753,0.580822
3,C_ID_d27d835a9f,0,1,0,0,0,1,0,0,0,0.008219,0.375342
4,C_ID_2b5e3df5c2,0,0,0,0,1,1,0,0,1,1.093151,2.271233


In [5]:
df_treino['target'] = df_treino['alvo']
df_treino.drop('alvo', axis = 1, inplace = True)

In [6]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 13 columns):
card_id              201917 non-null object
feature_1_1          201917 non-null int64
feature_1_2          201917 non-null int64
feature_1_3          201917 non-null int64
feature_1_4          201917 non-null int64
feature_1_5          201917 non-null int64
feature_2_1          201917 non-null int64
feature_2_2          201917 non-null int64
feature_2_3          201917 non-null int64
feature_3_1_0        201917 non-null int64
oldest_Activation    201917 non-null float64
latest_Activation    201917 non-null float64
target               201917 non-null float64
dtypes: float64(3), int64(9), object(1)
memory usage: 20.0+ MB


In [7]:
# Preparar os dados para normalização e ML
colunas = ['feature_1_1', 'feature_1_2', 'feature_1_3', 'feature_1_4',
       'feature_1_5', 'feature_2_1', 'feature_2_2', 'feature_2_3',
       'feature_3_1_0', 'oldest_Activation', 'latest_Activation', 'target']

dados = df_treino[colunas]
array = dados.values

# Separando o array em componentes de input (X) e output (Y)
X = array[:,0:11]
Y = array[:,11]

In [11]:
# Tunning modelo Ridge

# Definindo os valores que serão testados
#valores_grid = {'alpha': uniform()}
seed = 7

# Criando o modelo
modelo = Ridge()
iterations = 50000
rsearch = RandomizedSearchCV(estimator = modelo, 
                             param_distributions = valores_grid, 
                             n_iter = iterations, 
                             random_state = seed)
rsearch.fit(X, Y)

# Print do resultado
print("Melhores Parâmetros do Modelo:\n", rsearch.best_estimator_)

/home/angelo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [8]:
# Normalizar os dados
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Normalizando os dados
scaler    = MinMaxScaler(feature_range = (0, 1))
rescaledX = scaler.fit_transform(X)

# Seleção de atributos em 4 componentes (grupo de atributos com variância similar)
pca = PCA(n_components = 4)
fit = pca.fit(rescaledX)

In [9]:
# Garante que os resultados podem ser reproduzidos
# Isso é importante para comparar a acurácia com outros algoritmos de Machine Learning.
seed = 7
teste_size = 0.25

In [10]:
# Regressão Linear
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criação do modelo
modelo = LinearRegression()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 3.8719267697148507


In [11]:
# Ridge Regression
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criação do modelo
modelo = Ridge()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 3.87193818135416


In [12]:
# Lasso Regression
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modelo = Lasso()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 3.8743805476960786


In [13]:
# ElasticNet Regression
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modelo = ElasticNet()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 3.8743805476960786


In [ ]:
# KNN
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modelo = KNeighborsRegressor()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 4.2440930962945655


In [ ]:
# Cart
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modelo = DecisionTreeRegressor()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

O RMSE do modelo é: 5.440678022917314


In [ ]:
# SVM
# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modelo = SVR()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Fazendo previsões
Y_pred = modelo.predict(X_teste)

# Score do modelo nos dados de teste
rmse = math.sqrt(mean_squared_error(Y_teste, Y_pred))
print("O RMSE do modelo é:", rmse)

/home/angelo/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
